In [ ]:
%run "./Getting started.ipynb"

In [1]:
from emergent.utilities.decorators import experiment

@experiment
def artiq(self, state, params = {}):
    self.actuate(state)
    submit = {'pid': str(uuid.uuid1()), 'sequence': self.children['sequencer'].steps}
    submit['adc'] = [{'start': 10e-3, 'end': 20e-3, 'step': 100e-6, 'channels': [0]},
                     {'start': 40e-3, 'end': 60e-3, 'step': 100e-6, 'channels': [0, 1]}]
    requests.post('http://localhost:5000/artiq/run', json=submit)
    self.network.artiq_client.emit('submit')

    while True:
        response = requests.get('http://localhost:5000/artiq/run').json()
        if 'result' in response:
            break
    print('Received from ARTIQ:', response['result'])

    return 0

hub = network.hubs['hub']
hub.artiq = artiq

NameError: name 'experiment' is not defined

The experimental timeline is represented by a list of steps represented by dictionaries of the following format:

In [3]:
step = {'name': 'loading',
        'duration': 0.5,
        'TTL': [0, 3, 4],
        'ADC': [1, 3],
        'DAC': {1: 3.3, 2: 0.5},
        'DDS': {0: 400, 1: 250}
       }

The name and duration are self-explanatory. The TTL field specifies which TTL channels are on during this timestep; similarly, the ADC field specifies which channels should be acquiring during this timestep. The DAC and DDS fields contain both channel labels and their values.